In [1]:
# Thanks to 
# [Ramsri Goutham Golla](https://github.com/ramsrigouthamg) 
# [Suraj Patil](https://github.com/patil-suraj) 
# [renatoviolin](https://github.com/renatoviolin/Multiple-Choice-Question-Generation-T5-and-Text2Text)
# for their T5 transformer, BERT and QG codes. Their notebooks were instrumental in crafting this. 

In [2]:
#!pip install torch torchvision
#!pip install spacy==2.1.3 --upgrade --force-reinstall
#!pip install -U nltk
#!python -m spacy download en/de

#!pip install gensim
#!pip install git+https://github.com/boudinfl/pke.git !!!!!
#!pip install bert-extractive-summarizer --upgrade --force-reinstall !!!
#!pip install -U pywsd !!!
#!pip install -U wn==0.0.23 !!!
#!pip install flashtext!!!


In [3]:
import transformers
import torch
import nltk
from summarizer import Summarizer
from transformers import *
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords
import spacy
import requests
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from nltk.corpus import wordnet as wn

from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

nltk.download('stopwords')
nltk.download('popular')



Warming up PyWSD (takes ~10 secs)... took 6.344625234603882 secs.
[nltk_data] Downloading package stopwords to /home/jboeck/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jboeck/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/jboeck/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/jboeck/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/jboeck/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/jboeck/nltk_data...
[nltk_data]    |   Package

True

# Load Text and define Path 

In [4]:
bspText= "Igel.txt"
#bspText= "Waldtiere.txt"
#bspText= "Recycling_Kap_3_2.txt" 
#bspText= "ORF_Elefant.txt"
#bspText= "Krauter_2.txt"
#bspText= "Krauter.txt"
#bspText= "thermodynamik.txt"
#bspText= "Pflanzen.txt"

f = open(bspText,"r")
full_text = f.read()
path = "/home/workdrive/jboeck/Applications/QG/BAC/"





## BERT Extractive Summarizer
Summarize the text using BERT Extractive Summarizer. This is used to extract the most relevant and useful sentences from the input text.

In [5]:
#__BERT for text Summarization 
custom_config = AutoConfig.from_pretrained('distilbert-base-multilingual-cased') 
# may use other BERTs in future settings:
    # bert-base-german-cased
    # distilbert-base-german-cased

custom_config.output_hidden_states=True # Whether or not the model should return all hidden-states.

custom_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
custom_model = AutoModel.from_pretrained('distilbert-base-multilingual-cased', config=custom_config) #use autoconfig
#print ("device ",device)
#model = model.to(device)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
model(full_text)
result = model(full_text, min_length=10, ratio = 0.6) #Ratio of sentences to summarize to from the original body. (default to 0.2)

summarized_text = ''.join(result)
print ("___________summarized_text______________")
print (summarized_text)
print("______________________________________________________________________________________________")
print ("_____________full_text___________________")
print (full_text)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#print(custom_tokenizer)

___________summarized_text______________
Wenn die Igel sich ein gutes
Fettpolster angefressen haben,
gehen sie schlafen. Zuerst ziehen
sich die Igelmännchen zurück, je
nach Witterung bereits Anfang Oktober. Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben. Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate. Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter. Manchmal sind Igel auch
für wenige Tage aktiv und verlassen ihr Nest. Das Erwachen tritt meist bei länger anhaltenden Außentemperaturen
um 10° C ein. Warum halten Igel eigentlich Winterschlaf? Sie bräuchten
sehr viel Energie, um ihre Körpertemperatur auf 34°C zu halten. Diese Energie bekommen sie durch ihre Nahrung.
______________________________________________________________________________________________
_____________full_text___________________
Wenn die Igel sich ein gutes
Fettpolster a

### Used for Thesis

#### DistilBERT - multilingual
___________summarized_text______________

Wenn die Igel sich ein gutes
Fettpolster angefressen haben,
gehen sie schlafen. Zuerst ziehen
sich die Igelmännchen zurück, je
nach Witterung bereits Anfang Oktober. Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben. Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate. Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter. Manchmal sind Igel auch
für wenige Tage aktiv und verlassen ihr Nest. Das Erwachen tritt meist bei länger anhaltenden Außentemperaturen
um 10° C ein. Warum halten Igel eigentlich Winterschlaf? Sie bräuchten
sehr viel Energie, um ihre Körpertemperatur auf 34°C zu halten. Diese Energie bekommen sie durch ihre Nahrung.

_________________________________________________________________________________________________________________
_________________________________________________________________________________________________________________

### For future settings

#### DistilBERT - german
___________summarized_text______________

Wenn die Igel sich ein gutes
Fettpolster angefressen haben,
gehen sie schlafen. Zuerst ziehen
sich die Igelmännchen zurück, je
nach Witterung bereits Anfang Oktober. Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben. Im Winterschlaf sind alle Lebensfunktionen des Igels auf
Sparflamme gesetzt. Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate. Manchmal sind Igel auch
für wenige Tage aktiv und verlassen ihr Nest. Das Erwachen tritt meist bei länger anhaltenden Außentemperaturen
um 10° C ein. Warum halten Igel eigentlich Winterschlaf? Igel sind mit ihrem
Stachelkleid ganz schlecht vor der Kälte geschützt. Diese Energie bekommen sie durch ihre Nahrung. Da es aber im
Winter für den Igel kaum Futter gibt, würde er einfach verhungern
und erfrieren.

#### German BERT
___________summarized_text______________

Wenn die Igel sich ein gutes
Fettpolster angefressen haben,
gehen sie schlafen. Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben. Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate. Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter. Manchmal sind Igel auch
für wenige Tage aktiv und verlassen ihr Nest. Das Einschlafen selbst dauert beim Igel rund 5-6 Stunden. Warum halten Igel eigentlich Winterschlaf? Igel sind mit ihrem
Stachelkleid ganz schlecht vor der Kälte geschützt. Sie bräuchten
sehr viel Energie, um ihre Körpertemperatur auf 34°C zu halten. Diese Energie bekommen sie durch ihre Nahrung. Da es aber im
Winter für den Igel kaum Futter gibt, würde er einfach verhungern
und erfrieren.

## Keyword Extraction
Get important keywords from the text and filter those keywords that are present in the summarized text.

In [6]:
def get_nouns_multipartite(text):
    out=[]
    
    nlp = spacy.load("de_core_news_sm")
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input = bspText, language = 'de')
    pos = {'NOUN'} # just use nouns as keywords
    
    stoplist = list(string.punctuation)
    stoplist += stopwords.words('german') 

    
    extractor.candidate_selection(pos=pos, stoplist=stoplist)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=10)

    for key in keyphrases:
        out.append(key[0])
    return out

keywords = get_nouns_multipartite(full_text) 
filtered_keys=[] # just the keys in the summarized text

for keyword in keywords:
    keyword = keyword.capitalize()
    if keyword in summarized_text: 
        filtered_keys.append(keyword)


## Sentence Mapping
For each keyword get the sentences from the summarized text containing that keyword. 

In [7]:
keywords = filtered_keys

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 10 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 10]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        word = word.strip()
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values

    delete_keys = []
    for k in keyword_sentences.keys():
        if len(keyword_sentences[k]) == 0:
            delete_keys.append(k)
    for del_key in delete_keys:
        del keyword_sentences[del_key]
    return keyword_sentences





sentences = tokenize_sentences(summarized_text) #!!!
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)

print("___10 Best Sentences___")
print("\n")
index_number = 1
for sentence in sentences:
    
    print("Nr.",index_number,':\n',sentence,'\n')
    index_number= index_number + 1 
print('_______________________________________________________')



___10 Best Sentences___


Nr. 1 :
 Wenn die Igel sich ein gutes
Fettpolster angefressen haben,
gehen sie schlafen. 

Nr. 2 :
 Zuerst ziehen
sich die Igelmännchen zurück, je
nach Witterung bereits Anfang Oktober. 

Nr. 3 :
 Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben. 

Nr. 4 :
 Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate. 

Nr. 5 :
 Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter. 

Nr. 6 :
 Manchmal sind Igel auch
für wenige Tage aktiv und verlassen ihr Nest. 

Nr. 7 :
 Das Erwachen tritt meist bei länger anhaltenden Außentemperaturen
um 10° C ein. 

Nr. 8 :
 Warum halten Igel eigentlich Winterschlaf? 

Nr. 9 :
 Sie bräuchten
sehr viel Energie, um ihre Körpertemperatur auf 34°C zu halten. 

Nr. 10 :
 Diese Energie bekommen sie durch ihre Nahrung. 

_______________________________________________________


## Additional Distractor generation 
####  For MCQ Questions, not used in this thesis

In [8]:
#Possible Distractors not used within the thesis

# def get_wordsense(sent,word):
#     word= word.lower()
#     if len(word.split())>0:
#         word = word.replace(" ","_")   
#     synsets = wn.synsets(word,'n')
#     if synsets:
#         wup = max_similarity(sent, word, 'wup', pos='n')
#         adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
#         lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
#         return synsets[lowest_index]
#     else:
#         return None

    
# #German Distractors using Conceptnet   
# def get_distractors_conceptnet(word):
#     word = word.lower()
#     original_word= word
#     if (len(word.split())>0):
#         word = word.replace(" ","_")
#     distractor_list = [] 
#     #url = "http://api.conceptnet.io/query?filter=/c/de/node=/c/de/%s/n&rel=/r/PartOf&start=/c/de/%s&limit=5"%(word,word)
#     url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/IsA&start=/c/de/%s&limit=5"%(word,word)
#     #url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/TypeOf&start=/c/de/%s&limit=5"%(word,word)
#     #url = "http://api.conceptnet.io/query?node=/c/de/%s/n&rel=/r/RelatedTo&start=/c/de/%s&limit=5"%(word,word)
#     obj = requests.get(url).json()
   
#     for edge in obj['edges']:
#         link = edge['end']['term'] 
#         #url2 = "http://api.conceptnet.io/query?filter=/c/de/node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
#         url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/IsA&end=%s&limit=10"%(link,link)
#         #url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/TypeOf&end=%s&limit=10"%(link,link)
#         #url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/RelatedTo&end=%s&limit=10"%(link,link)
#         obj2 = requests.get(url2).json()
#         for edge in obj2['edges']:
#             word2 = edge['start']['label']
           
#             if word2 not in distractor_list and original_word.lower() not in word2.lower():
#                 distractor_list.append(word2)
#     return distractor_list




# key_distractor_list = {}
# # print(keyword_sentence_mapping)

# for keyword in keyword_sentence_mapping: 
#     wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword) 
#     if wordsense:
#         if len(distractors) ==0:
#             distractors = get_distractors_conceptnet(keyword)
#         if len(distractors) != 0:
#             key_distractor_list[keyword] = distractors
#     else:
#         distractors = get_distractors_conceptnet(keyword)
#         if len(distractors) != 0:
#             key_distractor_list[keyword] = distractors   



In [9]:
print("KEYWORDS: ", keywords)
print('_________________')

for keyword in keywords:
    print('\n')
    print(keyword_sentence_mapping[keyword][0])
    


KEYWORDS:  ['Igel', 'Winterschlaf', 'Monate', 'Nest', 'Energie', 'Dauer']
_________________


Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter.


Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
ausreichendes Winterschlafgewicht erreicht haben.


Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate.


Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter.


Sie bräuchten
sehr viel Energie, um ihre Körpertemperatur auf 34°C zu halten.


Die durchschnittliche Dauer des Winterschlafs beträgt 5 bis 6
Monate.


# Input Preparation


In [10]:
trained_model_path = path + 't5/model/'
trained_tokenizer =  path + 't5/tokenizer/'

model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


## load finetuned T5 Model

In [11]:
def prepare_inputs_for_qg_from_answers_hl(sents, answer):
        inputs = []
        sents = sents
        #print(sents)
        sents_copy = sents[:]
        answer_copy = answer[:]
        answer_text = answer_copy.strip()
        answer_text = "".join(answer_text)
        answer_text = answer_text.capitalize()
        #print("ANSWER TEXT:", answer_text)
        ans_start_idx = sents.index(answer_text)
#         print(ans_start_idx)      
#         print(type(sents))
#         print(len(answer_text))
        generate="generate question: "
        contextstr="context: "
        answerstr="answer: "
        
        #Experiments
        #sent = f"{generate}{sents[:ans_start_idx]} <hl> {answer_text} <hl> {sents[ans_start_idx + len(answer_text): ]}</s>"
        #sent = f"{generate}{sents[:ans_start_idx]} <hl> {answer_text} <hl> {sents[ans_start_idx + len(answer_text): ]}"
        #sent = f"{sents[:ans_start_idx]} <hl>{answer_text}<hl> {sents[ans_start_idx + len(answer_text): ]}"
        #sent = f"{contextstr}{sents[:ans_start_idx]}<hl>{answer_text}<hl>{sents[ans_start_idx + len(answer_text): ]}</s><hl>{answerstr}{answer_text}<hl>" #USED FOR BAC
        sent = f"{contextstr}{sents[:ans_start_idx]}{answer_text}{sents[ans_start_idx + len(answer_text): ]}<hl>{answerstr}{answer_text}<hl></s>"
        #sent = f"{contextstr}{sents[:ans_start_idx]}<hl>{answer_text}<hl>{sents[ans_start_idx + len(answer_text): ]}{answerstr}<hl>{answer_text}<hl>"
        #sent = f"{generate}{sents[:ans_start_idx]} <hl> {answer_text} <hl> {sents[ans_start_idx + len(answer_text): ]}</s><hl>{answerstr}{answer_text}<hl>"
        #sent = f"{answerstr}<hl>{answer_text}<hl> {contextstr}{sents[:ans_start_idx]}<hl>{answer_text}<hl>{sents[ans_start_idx + len(answer_text): ]}"
        
        #sents_copy = sents
        #print("sent", sent) 
        #print("sents_copy", sents_copy)
        #print("SENT", sent)
        #print("SENTS_COPY", sents_copy)
            
#         source_text = "".join(sents_copy)
#         source_text = f"generate question: {source_text}" 
#         print(type(source_text))
#         source_text = source_text + " </s>"
               
        #inputs.append({"answer": answer_text, "source_text": sent})
#                 #print(inputs)
        #inputs.append({ "generate questions": sent})
        #return sents_copy
        return sent 

# Question Generation with T5


In [22]:
def t2t_qg_with_t5(Q_numb,context,answer):
    print("\n")
    #Q_numb; 
    text = context
    answer = answer.capitalize()
    print("INPUT:", text)
    print("\n")
    encoding = tokenizer.encode_plus(text,max_length =512, padding=True, return_tensors="pt")
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    

    model.eval()
    beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=72,
    temperature = 0.05, #amount of randomness #
    early_stopping=True,
    num_beams=30, #beam: number of pads that we consider 
    num_return_sequences=10 
)   
    print("All_Question_Choices")
    for beam_output in beam_outputs:
        Q_numb= Q_numb + 1
        question = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        #print(type(question))
        question = " ".join(question.split()[1:]) #remove first "question:" -string
        print (str(Q_numb) +":",question)
    return question


                

# Generate Questions

In [21]:
print('______Questionaire______________________________')
print('_____________________________________________________________'+ "\n")

#print(key_distractor_list)
index = 1
Q_numb = 0
for keyword in keywords:   
    sentence = keyword_sentence_mapping[keyword][0] # get sentence for keyword
        
    #keyword =keyword.capitalize()
    #print(keyword)
    #__Question Generaion___
    qg_examples = prepare_inputs_for_qg_from_answers_hl(sentence, keyword)   
    question_t5= t2t_qg_with_t5(Q_numb, qg_examples,keyword)
    #_numb = Q_numb+1
    
#____for Multiple Choice_______________ 
    #choices = [keyword.capitalize()] + key_distractor_list[keyword]
    #top4choices = choices[:4]
    #random.shuffle(top4choices)
#     optionchoices = ['a','b','c','d']
#     for idx,choice in enumerate(top4choices):
#         print ("\t",optionchoices[idx],")"," ",choice.capitalize())
#         #print ("\t",optionchoices[idx],")"," ",choice)
#     print ("\nMore options: ", choices[4:20],"\n\n")
#________________________________________________________________________
    index = index + 1 
    print("\n")
    

______Questionaire______________________________
_____________________________________________________________



INPUT: context: Kurze Unterbrechungen sind normal, die Igel erwachen,
bleiben im Nest und schlafen bald weiter.<hl>answer: Igel<hl></s>


All_Question_Choices
1: Was bleibt im Nest und schlafen bald?
2: Was bleiben im Nest und schlafen bald?
3: Wer bleibt im Nest und schlaft bald?
4: Wie erwachen Kurze Unterbrechungen?
5: Wer bleibt im Nest und schlafen bald?
6: Was bleibt im Nest und schlaft bald?
7: Wer bleibt im Nest und schlaft bald?
8: Was bleibt normal im Nest und schlafen bald?
9: Wer bleibt normal im Nest und schlaft bald?
10: Was bleibt im Nest und schlafen bald?
11: Was bleibt im Nest und schlafen bald weiter?
12: Wer bleiben im Nest und schlafen bald?
13: Was bleibt im Nest?
14: Wer bleibt im Nest und schlaft bald weiter?
15: Wer bleibt im Nest und schlafen bald?




INPUT: context: Zuletzt
gehen die Jungigel in Winterschlaf, sie brauchen ihre Zeit, bis sie ein
a

In [14]:
tokenizer.eos_token




'</s>'

In [15]:
tokenizer.decode([0])


'<pad>'

In [16]:
tokenizer.decode([1])


'</s>'

In [17]:
tokenizer.decode([2])

'<unk>'

In [18]:
tokenizer.decode([32101])

'<hl>'

In [19]:
tokenizer.decode([32100])

'<sep>'